<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Transformers_for_Time_Series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

# Generate synthetic time series data
def generate_time_series(n_series, n_timepoints):
    t = np.linspace(0, 50, n_timepoints)
    data = np.sin(t) + np.random.normal(0, 0.1, (n_series, n_timepoints))
    return data

n_series = 100
n_timepoints = 200
data = generate_time_series(n_series, n_timepoints)

# Prepare data for the Transformer model
def prepare_data(data, input_len, output_len):
    x, y = [], []
    for i in range(data.shape[1] - input_len - output_len):
        x.append(data[:, i:i+input_len])
        y.append(data[:, i+input_len:i+input_len+output_len])
    return np.array(x), np.array(y)

input_len = 20
output_len = 10
x, y = prepare_data(data, input_len, output_len)
x = torch.FloatTensor(x)  # x shape: [n_samples, n_series, input_len]
y = torch.FloatTensor(y)  # y shape: [n_samples, n_series, output_len]

# Transpose x and y to [input_len, batch_size, n_series]
x = x.permute(2, 0, 1)  # x shape: [input_len, n_samples, n_series]
y = y.permute(2, 0, 1)  # y shape: [output_len, n_samples, n_series]

# Define the Transformer model
class TransformerTimeSeries(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_heads, dropout):
        super(TransformerTimeSeries, self).__init__()
        self.input_projection = nn.Linear(input_size, hidden_size)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_size, nhead=num_heads, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(hidden_size, input_size)

    def forward(self, src):
        print("Input shape before projection:", src.shape)  # Debug
        src = self.input_projection(src)  # src shape: [sequence_length, batch_size, hidden_size]
        print("Shape after projection:", src.shape)  # Debug
        src = self.transformer_encoder(src)
        output = self.fc(src)
        return output

# Verify shape
print("x shape:", x.shape)
print("y shape:", y.shape)

# Set input_size to the number of series
input_size = n_series  # Number of time series
model = TransformerTimeSeries(input_size=input_size, hidden_size=64, num_layers=2, num_heads=4, dropout=0.1)

# Train the model
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(100):
    model.train()
    optimizer.zero_grad()
    output = model(x[:output_len])  # Ensure the output has the same sequence length as the target
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")